In [1]:
from sqlalchemy import create_engine
import pandas as pd

## Load Data
We have two tables in the database:
- rides with ride data
- stations with station data (lng/lat)

In [3]:
sql = '''
SELECT
	r.trip_id,
	r.bikeid AS bike_id,
	r.subscriber_type,
	r.checkout_time,
	r.duration_minutes,
	r.start_station_id,
	r.start_station_name,
	s.latitude AS start_latutide,
	s.longitude AS start_longitude,
	r.end_station_id,
	e.latitude AS end_latutide,
	e.longitude AS end_longitude
FROM 
	rides r 
		LEFT JOIN stations s ON r.start_station_id == s.station_id
		LEFT JOIN stations e ON r.end_station_id == e.station_id
;
'''

conn = create_engine('duckdb:///../../data/bikes.ddb').connect()
df = pd.read_sql(sql, conn)
print('size:', len(df))

size: 649231


## Clean Data


In [4]:
mask = (
    (df['duration_minutes'] > 0) &
    ~pd.isnull(df['bike_id']) &
    ~pd.isnull(df['start_station_id']) &
    ~pd.isnull(df['end_station_id'])
)
df = df[mask]

df['bike_id'] = df['bike_id'].astype('int64')
df['start_station_id'] = df['start_station_id'].astype('int64')
df['end_station_id'] = df['end_station_id'].astype('int64')

print('size:', len(df))

size: 603770


## Run Calculations

In [5]:
print('longest duration')
(
    df.groupby('bike_id', as_index=False)
    ['duration_minutes'].sum()
    .iloc[-1]
)

longest duration


bike_id             5089
duration_minutes    3914
Name: 409, dtype: int64